# Vehicle Detection, Tracking and Speed Estimation

This notebook detects, tracks, and estimates the speed of vehicles on a highway using YOLOv8 and DeepSORT.

# Install necessary dependencies

In [ ]:
!pip install ultralytics deep-sort-realtime opencv-python-headless

import os
import cv2
import math
import torch
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Ensure GPU is available

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Define video path and create output directory

In [ ]:
video_path = "highway_video.mp4"
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video not found at {video_path}")

# Load video

In [ ]:
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(os.path.join(output_dir, "output.mp4"), fourcc, fps, (width, height))


# Load YOLOv8 model

In [ ]:
model = YOLO("yolov8n.pt").to(device)

# Initialize DeepSORT tracker

In [ ]:
tracker = DeepSort(max_age=15, n_init=5, nn_budget=100, max_cosine_distance=0.25)

# Speed estimation parameters

In [ ]:
pixel_to_meter = 0.0461  # Calibration required
speed_limit_kmh = 30  # Speed limit in km/h
car_positions = {}

# Process video frame by frame

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame, stream=True)
    detections = []
    for result in results:
        for box in result.boxes.cpu().numpy():
            class_id = int(box.cls[0])
            conf = float(box.conf[0])
            if result.names[class_id] == 'car' and conf > 0.6:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                detections.append(((x1, y1, x2 - x1, y2 - y1), conf, 'car'))

    # Track detected vehicles
    if detections:
        tracks = tracker.update_tracks(detections, frame=frame)
        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

            # Calculate speed
            if track_id in car_positions:
                prev_x, prev_y = car_positions[track_id]
                distance_pixels = math.sqrt((center_x - prev_x)**2 + (center_y - prev_y)**2)
                distance_meters = distance_pixels * pixel_to_meter
                time_seconds = 1 / fps
                speed_kmh = (distance_meters / time_seconds) * 3.6
            else:
                speed_kmh = 0

            # Display speed and bounding box
            color = (0, 255, 0) if speed_kmh <= speed_limit_kmh else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"ID: {track_id} Speed: {speed_kmh:.2f} km/h", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Store position for next frame
            car_positions[track_id] = (center_x, center_y)

    # Save processed frame
    out.write(frame)


# Release resources

In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. Video saved in results/output.mp4")

# Video result: https://www.youtube.com/watch?v=QpQxzGtMqsk
